In [2]:
import pandas as pd
import hvplot.pandas
import panel as pn
import numpy as np
pn.extension('tabulator')
from transformers import AutoModelForSequenceClassification
from transformers import  AutoTokenizer
from scipy.special import softmax
import torch

In [3]:
data = pd.read_csv('datajm_3.csv')

In [4]:
data

,Year,Country,% Change in Labor Input,% Change in Nutrient Balance,% Change in Production
0,1991,Ireland,-10.676569,0.000000,-3.959618
1,1992,Ireland,-1.067887,0.000000,8.603737
2,1993,Ireland,-2.852737,13.207547,-0.447907
3,1994,Ireland,-3.055556,11.666667,3.585840
4,1995,Ireland,-5.157593,7.462687,1.590633
...,...,...,...,...,...
76,2013,France,-0.913474,26.380000,-3.130000
77,2014,France,-0.832266,-0.340000,1.550000
78,2015,France,-1.665591,26.010000,-0.160000
79,2016,France,-1.141019,27.350000,-6.280000


In [5]:
#Making DataFrame Pipeline Interactive 
dd =data.interactive()

In [6]:
#Define Panel Widgets
yearslider = pn.widgets.IntSlider (name ='year slider', start=1991, end=2017, step=3,value=2001)

In [7]:
#Radio button for Production
yaxis_prod = pn.widgets.RadioButtonGroup(
name= 'Y Axis',
    options=['% Change in Production',],
    button_type='success'
)

In [8]:
#Production Pipeline
prod_pipeline =(
dd[(dd.Year <=yearslider)]
    .groupby (['Country', 'Year']) [yaxis_prod].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [9]:
#Production Plot
Prodplot =prod_pipeline.hvplot(x='Year', by='Country', y=yaxis_prod, line_width=2, title="% Change in Production by Country")
Prodplot

In [10]:
prod_table = prod_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')
prod_table

In [11]:
#Radio button for Nutrient Balance
yaxis_nb = pn.widgets.RadioButtonGroup(
name= 'Y Axis',
    options=['% Change in Nutrient Balance',],
    button_type='success'
)

In [12]:
#Nutrient Balance Pipeline
nb_pipeline =(
dd[(dd.Year <=yearslider)]
    .groupby (['Country', 'Year']) [yaxis_nb].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [13]:
#Nutrient Balance Plot
nbplot =nb_pipeline.hvplot(x='Year', by='Country', y=yaxis_nb, line_width=2, title="% Change in Nutrient Balance by Country")
nbplot

In [14]:
#Nutrient Balance Table
nbtable = nb_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')
nbtable

In [15]:
#Radio button for Labor Input
yaxis_li = pn.widgets.RadioButtonGroup(
name= 'Y Axis',
    options=['% Change in Labor Input',],
    button_type='success'
)

In [16]:
#Labor Input Pipeline
li_pipeline =(
dd[(dd.Year <=yearslider)]
    .groupby (['Country', 'Year']) [yaxis_li].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [17]:
#Labor Input Plot
liplot =li_pipeline.hvplot(x='Year', by='Country', y=yaxis_li, line_width=2, title="% Change in Labor Input by Country")
liplot

In [18]:
#Labor Input Table
litable = li_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')
litable

In [19]:
#ScatterPlot 1 Pipeline: Production against Nutrient Balance
prodnbpipeline=(
dd[(dd.Year == yearslider)]
    .groupby(['Country', 'Year', '% Change in Nutrient Balance'])['% Change in Production'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [20]:
prodnbscatterplot = prodnbpipeline.hvplot(x='% Change in Nutrient Balance', y='% Change in Production',by='Country', alpha=0.7, size=20, kind="scatter",
                                          title="A Scatter Plot of Production against Nutrient Balance", legend=False, height=500, width=250)
prodnbscatterplot

In [21]:
#ScatterPlot 1 Pipeline: Production against Labor Input
prodlipipeline=(
dd[(dd.Year == yearslider)]
    .groupby(['Country', 'Year', '% Change in Labor Input'])['% Change in Production'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [22]:
prodliscatterplot = prodlipipeline.hvplot(x='% Change in Labor Input', y='% Change in Production', title="A Scatter Plot of Production against Labor Input", by='Country', size=20, kind="scatter", alpha=0.7, legend=False, height=500, width=250)
prodliscatterplot

In [23]:
#Creatig the Dashboard
template=pn.template.FastListTemplate(
title='Impacts of Nutrient Balance and Labor Input on Agricultural Production: Case of Ireland, Spain and France',
    sidebar=
    [pn.pane.Markdown("# Impacts of Nutrient Balance and Labor Input on Agricultural Production: Case of Ireland, Spain and France"),
    pn.pane.Markdown("### After Conducting Regression data analysis on EU agricultural sector in data for Ireland, Spain, and France, Nutrient Balance and Labor Input were founf to have insginificant impact of Agricultural Production"),
    pn.pane.PNG('datajm.png', sizing_mode='scale_both'),
    pn.pane.Markdown("# settings"), yearslider],
main= [pn.Row(pn.Column(yaxis_prod, Prodplot.panel(width=500),margin=(0,25)),
        prod_table.panel(width=500)),
        pn.Row(pn.Column(yaxis_nb, nbplot.panel(width=500),margin=(0,25)),
       nbtable.panel(width=500)),
       pn.Row(pn.Column(yaxis_li, liplot.panel(width=500),margin=(0,25)),
        litable.panel(width=500)),
       pn.Row(pn.Column(prodnbscatterplot.panel(width=500), margin=(0,25)), 
              pn.Column(prodliscatterplot.panel(width=500), margin=(0,25)))],
  accent_base_color="#88d8b0",
header_background="#88d8b0",)
template.show()
template.servable();


Launching server at http://localhost:63667


In [24]:
#WSentment Analysis


In [25]:
tweet = "@eu Nutrient Balance and Labor Input do not significantly impact Agricultural Prosuction in EU"


In [26]:
#process tweer
tweet_words =[]
for word in tweet.split('spaces'):
    if word.startswith('@') and len(word)> 1:
        word ='@user'
tweet_proc =''.join(tweet_words)
        

In [27]:
#tweet processing
tweet_proc = " ".join(tweet_words)


In [28]:
#Load model and Tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification. from_pretrained(roberta)
tokenizer=AutoTokenizer.from_pretrained(roberta)
lebles= ['Negative', 'Neutral','Positive']

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

C:\Users\jmdjo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jmdjo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [29]:
#Sentiment Analysis
encoded_tweet =tokenizer(tweet_proc, return_tensors='pt')
output = model(**encoded_tweet)
scores =output[0][0].detach().numpy()
scores = softmax(scores)

In [30]:
output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])


In [31]:
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1757,  0.3822, -0.0584]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [32]:
print(scores)

[0.2582943 0.4512725 0.2904333]
